In [33]:
import pandas as pd
from pandas.errors import ParserError
import pickle
import re

In [34]:
class CustomerDataExtractor:
    """
    A class to extract and flatten nested customer order data into a clean pandas DataFrame.

    Attributes:
        customer_orders (list): List of customer records.
        vip_customers (set): Set of VIP customer IDs.
    """

    def __init__(self, customer_orders, vip_customers):
        """
        Initializes the extractor with customer and VIP data.

        Args:
            customer_orders (list): Nested customer data loaded from .pkl file.
            vip_customers (set): Set of VIP customer IDs loaded from .txt file.
        """
        self.customer_orders = customer_orders
        self.vip_customers = vip_customers
        self.category_map = {
            1: 'Electronics',
            2: 'Apparel',
            3: 'Books',
            4: 'Home Goods'
        }

    def flatten_data(self):
        """
        Flattens nested customer order data into a DataFrame with the required format.

        Returns:
            pd.DataFrame: Flattened and sorted customer order data.
        """
        flat_data = []

        for customer in self.customer_orders:
            try:
                customer_id = customer.get('id')
                customer_name = customer.get('name')
                registration_date = pd.to_datetime(customer.get('registration_date'), errors='raise')
                is_vip = customer_id in self.vip_customers
            except (ParserError, ValueError):
                continue

            for order in customer.get('orders', []):
                try:
                    order_id = order.get('order_id')
                    if isinstance(order_id, str) and order_id.upper().startswith("ORD"):
                        order_id = int(order_id[3:])
                    order_date = pd.to_datetime(order.get('order_date'), errors='raise')
                except (ParserError, ValueError, TypeError):
                    continue

                items = order.get('items', [])
                if not items:
                    continue

                total_order_value = 0
                item_totals = []
                for item in items:
                    try:
                        price = item['price']
                        if isinstance(price, str):
                            price = float(price.replace('$', '').replace(',', ''))
                        quantity = int(item['quantity'])
                        total_price = price * quantity
                        item_totals.append((item, price, quantity, total_price))
                        total_order_value += total_price
                    except (ValueError, TypeError, KeyError):
                        continue

                for item, price, quantity, total_price in item_totals:
                    category = self.category_map.get(item.get('category'), 'Misc')
                    percentage = (total_price / total_order_value) * 100 if total_order_value else 0

                    flat_data.append({
                        "customer_id": customer_id,
                        "customer_name": customer_name,
                        "registration_date": registration_date,
                        "is_vip": is_vip,
                        "order_id": order_id,
                        "order_date": order_date,
                        "product_id": int(item.get('item_id')) if item.get('item_id') is not None else pd.NA,
                        "product_name": item.get('product_name'),
                        "category": category,
                        "unit_price": round(float(price), 2),
                        "item_quantity": quantity,
                        "total_item_price": round(total_price, 2),
                        "total_order_value_percentage": round(percentage, 2)
                    })

        df = pd.DataFrame(flat_data)

        # Fix column types and clean
        df['customer_id'] = df['customer_id'].astype('Int64')
        df['order_id'] = df['order_id'].astype('Int64')  # Preserve <NA>
        df['product_id'] = df['product_id'].astype('Int64')
        df['registration_date'] = pd.to_datetime(df['registration_date'], errors='coerce')
        df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')

        # Recover missing order_id from product_name
        def recover_order_id(row):
            if pd.isna(row['order_id']) and isinstance(row['product_name'], str):
                match = re.search(r'Order (\d+)', row['product_name'])
                if match:
                    return int(match.group(1))
            return row['order_id']

        # Recover missing product_id from product_name
        def recover_product_id(row):
            if pd.isna(row['product_id']) and isinstance(row['product_name'], str):
                match = re.search(r'Item (\d+)', row['product_name'])
                if match:
                    return int(match.group(1))
            return row['product_id']

        df['order_id'] = df.apply(recover_order_id, axis=1).astype('Int64')
        df['product_id'] = df.apply(recover_product_id, axis=1).astype('Int64')

        # Recover product_name if missing and both IDs are available
        df['product_name'] = df.apply(
            lambda row: f"Item {row['product_id']} for Order {row['order_id']}"
            if pd.notna(row['product_id']) and pd.notna(row['order_id'])
            else row['product_name'],
            axis=1
        )

        df = df.sort_values(by=["customer_id", "order_id", "product_id"])
        return df

=== USAGE EXAMPLE ===

In [35]:
# Load .pkl file
with open("customer_orders.pkl", "rb") as f:
    customer_orders = pickle.load(f)

In [36]:
# Load VIP list from txt
with open("vip_customers.txt", "r") as f:
    vip_ids = set(map(int, f.read().splitlines()))

In [37]:
# Create extractor and flatten data
extractor = CustomerDataExtractor(customer_orders, vip_ids)
df = extractor.flatten_data()

In [38]:
# Check the data types
print(df.dtypes)

customer_id                              Int64
customer_name                           object
registration_date               datetime64[ns]
is_vip                                    bool
order_id                                 Int64
order_date                      datetime64[ns]
product_id                               Int64
product_name                            object
category                                object
unit_price                             float64
item_quantity                            int64
total_item_price                       float64
total_order_value_percentage           float64
dtype: object


In [39]:
# Export to CSV or inspect
df.to_csv("cleaned_output.csv", index=False)
print(df.head())

   customer_id customer_name   registration_date  is_vip  order_id  \
0            1    Customer 1 2022-12-31 04:19:19    True         2   
1            1    Customer 1 2022-12-31 04:19:19    True         2   
2            1    Customer 1 2022-12-31 04:19:19    True         2   
3            1    Customer 1 2022-12-31 04:19:19    True         3   
4            2    Customer 2 2022-05-27 00:23:28   False         5   

           order_date  product_id        product_name     category  \
0 2025-05-03 08:09:20           1  Item 1 for Order 2   Home Goods   
1 2025-05-03 08:09:20           2  Item 2 for Order 2         Misc   
2 2025-05-03 08:09:20           3  Item 3 for Order 2         Misc   
3 2023-09-06 00:42:50           1  Item 1 for Order 3  Electronics   
4 2024-08-28 19:37:56           1  Item 1 for Order 5         Misc   

   unit_price  item_quantity  total_item_price  total_order_value_percentage  
0       52.70              2            105.40                          3.81  
